In [1]:
!pip install --user mwparserfromhell
!pip install --user subprocess32
!pip install --user xml-python

In [2]:
# Libraries
import xml.sax
import subprocess
import mwparserfromhell

In [3]:
# Function where ContentHandler looks for opening and closing tags title and text 
# and adds characters enclosed within them to the buffer
# content saved to a dict with tag as key

class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):         #do we need timestamp?
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [9]:
data_path = r"/home/mmartinelli/project/corpora/wikidumps/itwiki-20210720-pages-articles-multistream.xml.bz2"
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

# Iterating through compressed file
for i, line in enumerate(subprocess.Popen(['bzcat'], stdin = open(data_path), stdout = subprocess.PIPE).stdout):
    parser.feed(line)
    
    # Stop when n articles have been found
    if len(handler._pages) > 350:
        break
        


In [11]:
# Get titles of wiki articles
title_lst = [x[0] for x in handler._pages]
print(title_lst)

['Organo a pompa', 'Antropologia', 'Agricoltura', 'Architettura', 'Astronomia', 'Archeologia subacquea', 'Analisi delle frequenze', 'Aerofoni', 'Arte', 'Abbie Hoffman', 'Adige', 'Alto Adige', 'Accordo De Gasperi-Gruber', 'Anni 1950', 'Anni 1960', 'Anni 1970', 'Anni 1980', 'Anni 1990', 'Anni 2000', 'Anni 1940', 'Anni 1930', 'Anni 1920', 'Anni 1910', 'Anni 1900', 'Anni 1890', 'Anni 1880', 'Anni 1870', 'Anni 1860', 'Anni 1850', 'Anni 1840', 'Anni 1830', 'Anni 1820', 'Anni 1810', 'Anni 1800', 'Lista di astronauti e cosmonauti', 'Anni 1790', 'Anni 1780', 'Anni 1770', 'Anni 1760', 'Anni 1260', 'Anni 1300', 'Anni 1310', 'Anni 1320', 'Anni 1330', 'Anni 1340', 'Anni 1350', 'Anni 1360', 'Anni 1370', 'Anni 1380', 'Anni 1390', 'Anni 1750', 'Anni 1740', 'Anni 1730', 'Anni 1720', 'Anni 1710', 'Anni 1700', 'Alfred Nobel', 'Sport individuale', 'Anni 1290', 'Anni 1280', 'Anni 1270', 'Anni 1250', 'Anni 1240', 'Anni 1230', 'Anni 1220', 'Anni 1210', 'Anni 1200', 'Anni 1410', 'Anni 1420', 'Anni 1430', 'Ann

In [7]:
# Create the nth wiki article
wiki = mwparserfromhell.parse(handler._pages[305][1])    # [n][0] gets title tag, [n][1] gets text tag
print(type(wiki))

# Print first nth lines 
# WARNING: not a string but mwparserfromhell.wikicode.Wikicode object
print(wiki[:1000])

<class 'mwparserfromhell.wikicode.Wikicode'>
{{nd| il romanzo del 1962 scritto da [[Anthony Burgess]]|[[Arancia meccanica (romanzo)]]}} 
 {{Film 
 |titolo italiano = Arancia meccanica 
 |immagine = Clockwork Orange Trailer poster.png 
 |didascalia = Immagine del ''trailer'' in lingua inglese 
 |titolo originale = A Clockwork Orange 
 |lingua originale = [[Lingua inglese|inglese]], [[nadsat]] 
 |paese = [[Stati Uniti d'America]] 
 |paese 2 = [[Regno Unito]] 
 |anno uscita = [[1971]] 
 |durata = 136 min 
 |aspect ratio = 1,37:1 < small > (negativo) < /small > < br / > 1,66:1 < small > (cinema europei) < /small > < br / > 1,33:1 < small > (home video) < /small >  1,66:1 < small > (Blu-ray) < /small > 
 |genere = Grottesco 
 |genere 2 = Drammatico 
 |genere 3 = Thriller 
 |genere 4 = fantascienza 
 |regista = [[Stanley Kubrick]] 
 |soggetto = [[Anthony Burgess]]  < small > [[Arancia meccanica (romanzo)|(romanzo)]] < /small > 
 |sceneggiatore = Stanley Kubrick 
 |produttore = Stanley Kubric

In [8]:
# Print clean text
# WARNING: not a string but mwparserfromhell.wikicode.Wikicode object
wiki.strip_code().strip()[:1000]

"Arancia meccanica (A Clockwork Orange) è un film del 1971 scritto, prodotto e diretto da Stanley Kubrick. \n \n Tratto dall'omonimo romanzo distopico scritto da Anthony Burgess nel 1962, prefigura, appoggiandosi a uno stile sociologico e politico, una società votata a un'esasperata violenza, soprattutto nei giovani, e a un condizionamento del pensiero sistematico. \n \n Forte di quattro candidature agli Oscar del 1972 come miglior film, miglior regia, miglior sceneggiatura non originale e miglior montaggio, presentato lo stesso anno alla Mostra di Venezia, Decisivo per la riuscita del film fu anche l'apporto di Malcolm McDowell nel ruolo di Alex, pronto e disponibile a tutto, al punto che s'incrinò una costola e subì l'abrasione delle cornee durante le riprese del film. < ref >  < /ref > \n \n Quando fu distribuita sul circuito cinematografico, all'inizio degli anni settanta, la pellicola destò scalpore, con una schiera di ammiratori pronti a gridare al capolavoro, ma anche con una fo

In [24]:
import re

def process_article(title, text, timestamp, template = 'Film'):
    """Process a wikipedia article looking for template"""
    
    # Create a parsing object
    wikicode = mwparserfromhell.parse(text)
    
    # Search through templates for the template
    matches = wikicode.filter_templates(matches = template)
    
    # Filter out errant matches
    matches = [x for x in matches if x.name.strip_code().strip().lower() == template.lower()]
    
    if len(matches) >= 1:
        # template_name = matches[0].name.strip_code().strip()

        # Extract information from infobox
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in matches[0].params
                      if param.value.strip_code().strip()}

        # Extract internal wikilinks
        wikilinks = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]

        # Extract external links
        exlinks = [x.url.strip_code().strip() for x in wikicode.filter_external_links()]

        # Find approximate length of article
        text_length = len(wikicode.strip_code().strip())

        return (title, properties, wikilinks, exlinks, timestamp, text_length)

In [25]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Parse through XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._film = []
        self._article_count = 0
        self._non_matches = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text', 'timestamp'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._article_count += 1
            # Search through the page to see if the page is a book
            film = process_article(**self._values, template = 'Film')
            # Append to the list of books
            if film:
                self._film.append(film)

In [26]:
# Object for handling xml
handler = WikiXmlHandler()

# Parsing object
parser = xml.sax.make_parser()
parser.setContentHandler(handler)

for i, line in enumerate(subprocess.Popen(['bzcat'], 
                         stdin = open(data_path), 
                         stdout = subprocess.PIPE).stdout):
    parser.feed(line)
    
    # Stop when 3 articles have been found
    if len(handler._film) > 1:
        break
        
print(f'Searched through {handler._article_count} articles to find 3 films.')

Searched through 306 articles to find 3 films.
